In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor



# Load the dataset
passing_yds_table = pd.read_csv("NFL_Passing_STAT.csv")
#Essentially the plan is to create a scaling system for the years, create a new column called wiieght where we get a new passing yards value *weight
# # then group the data by player id, add the weights inot one number inside a new data frame and divide it by the number of years they have played, 
# then add that new score, plus the completion percentage, touchdown to interception ratio and completion percetage rating and qbr to see who is th best quarterback in the nfl

PreProcessing: We want to utilise A Weighted Aggregate Algorithm Using Linear decay on the weights for a player, to predict the future year numbers. Thus, we must grant weights to each of the years we have in the order being the most recent year must be the highest scaled value, proprtion of 1, the second highest scale should see a small difference, the third should see an equidistant difference from the 1st-2nd and the 2nd to the 3rd. Thus this follows for all other years a user would like to give

In [3]:
#The biggest factor that can alter the model is the fact taht we ahve multiple years with the same players
#THus we need to create a weighted scale that gives the most recent year the most say in the model
#create a dictionary that has the year associated with the weight 
year_to_scale= {
    "2023":1.0,
    "2022":0.75,
    "2021":0.5,
    "2020":0.25   
}
#passing_yds_table["Weights"] = passing_yds_table["Year"].map(year_to_scale)
type(passing_yds_table["Year"][0])#int64numpy
k = list(year_to_scale.keys())
v = list(year_to_scale.values())
type(k[v.index(1.0)])#str
#incompatible types, we need to make sure that the types are correct

#passing_yds_table["Weights"]
passing_yds_table["Weights"] = passing_yds_table["Year"].astype(str).map(year_to_scale)
passing_yds_table["Weights"]


0      1.00
1      1.00
2      1.00
3      1.00
4      1.00
       ... 
467    0.25
468    0.25
469    0.25
470    0.25
471    0.25
Name: Weights, Length: 472, dtype: float64

In [4]:
passing_yds_table[passing_yds_table["Player"] == "Matthew Stafford"]


,Player,Pass Yds,Cmp%,TD,INT,Rate,Year,Weights
10,Matthew Stafford,3965,62.6,24,11,92.5,2023,1.00
147,Matthew Stafford,2087,68.0,10,8,87.4,2022,0.75
234,Matthew Stafford,4886,67.2,41,17,102.9,2021,0.50
364,Matthew Stafford,4084,64.2,26,10,96.3,2020,0.25


In [8]:
#We need to account for retired players who have large enough numbers that could affect the model so that our model does not add them into the calculation
retired = ["Tom Brady","Drew Brees","Ben Roethlisberger","Matt Ryan","Cam Newton"]
#GO through the table O(n) check whether the player is inthe retired list, if they are, then we remove them out of the dataframe
count = 0
for i,x in passing_yds_table.iterrows():
    if x['Player'] in retired:
        count+=1
        #make sure that the shift in the dataframe doesent occur
        passing_yds_table.drop(i,inplace=True)
passing_yds_table[passing_yds_table["Player"] == "Tom Brady"]
#count


,Player,Pass Yds,Cmp%,TD,INT,Rate,Year,Weights,Player_id


In [6]:
#also create unique ids so that it is easy to group the players into one later o
passing_yds_table['Player_id'] = passing_yds_table['Player'].factorize()[0]
passing_yds_table.head()



,Player,Pass Yds,Cmp%,TD,INT,Rate,Year,Weights,Player_id
0,Tua Tagovailoa,4624,69.3,29,14,101.1,2023,1.0,0
1,Jared Goff,4575,67.3,30,12,97.9,2023,1.0,1
2,Dak Prescott,4516,69.5,36,9,105.9,2023,1.0,2
3,Josh Allen,4306,66.5,29,18,92.2,2023,1.0,3
4,Brock Purdy,4280,69.4,31,11,113.0,2023,1.0,4


In [7]:
passing_yds_table[passing_yds_table["Player_id"] == 1]

,Player,Pass Yds,Cmp%,TD,INT,Rate,Year,Weights,Player_id
1,Jared Goff,4575,67.3,30,12,97.9,2023,1.00,1
123,Jared Goff,4438,65.1,29,7,99.3,2022,0.75,1
251,Jared Goff,3245,67.2,19,8,91.5,2021,0.50,1
366,Jared Goff,3952,67.0,20,13,90.0,2020,0.25,1


In [11]:
passing_yds_table['weighted_yards'] = passing_yds_table['Pass Yds'] * passing_yds_table['Weights']
passing_yds_table.head()


,Player,Pass Yds,Cmp%,TD,INT,Rate,Year,Weights,Player_id,weighted_yards
0,Tua Tagovailoa,4624,69.3,29,14,101.1,2023,1.0,0,4624.0
1,Jared Goff,4575,67.3,30,12,97.9,2023,1.0,1,4575.0
2,Dak Prescott,4516,69.5,36,9,105.9,2023,1.0,2,4516.0
3,Josh Allen,4306,66.5,29,18,92.2,2023,1.0,3,4306.0
4,Brock Purdy,4280,69.4,31,11,113.0,2023,1.0,4,4280.0


In [ ]:
#Essentially. there exists a problem since when we look at a player whose play is decreasing, by adding the weight factor, the model might see that they could be increasing in play even though
#there play is decreasing since the faactors can divide their best year in half if it was two years ago, thus we need to add a difference column that takes in whether the player is increasing in 
#play or he is decreasing in play
passing_yds_table[passing_yds_table["Player_id"]==1]

,Player,Pass Yds,Cmp%,TD,INT,Rate,Year,Weights,Player_id,weighted_yards
1,Jared Goff,4575,67.3,30,12,97.9,2023,1.00,1,4575.0
123,Jared Goff,4438,65.1,29,7,99.3,2022,0.75,1,3328.5
251,Jared Goff,3245,67.2,19,8,91.5,2021,0.50,1,1622.5
366,Jared Goff,3952,67.0,20,13,90.0,2020,0.25,1,988.0


In [21]:
passing_yds_table['yearly_change'] = passing_yds_table.groupby('Player_id')['Pass Yds'].diff()
passing_yds_table[passing_yds_table["Player_id"]==1]

,Player,Pass Yds,Cmp%,TD,INT,Rate,Year,Weights,Player_id,weighted_yards,yearly_change
1,Jared Goff,4575,67.3,30,12,97.9,2023,1.00,1,4575.0,NaN
123,Jared Goff,4438,65.1,29,7,99.3,2022,0.75,1,3328.5,-137.0
251,Jared Goff,3245,67.2,19,8,91.5,2021,0.50,1,1622.5,-1193.0
366,Jared Goff,3952,67.0,20,13,90.0,2020,0.25,1,988.0,707.0
